# Data Mining

1. Display the collected data.

2. Deal with data and make it the csv file we want.
   1. 1 second time window


In [1]:
import pandas as pd
from datetime import datetime

In [2]:
cheatData = []
for ad in ['../raw_data/cheat_user1_1.log', '../raw_data/cheat_user1_2.log']:#, '../data/normal_user1_1.log']:
    f = open(ad)
    cheatData+=f.readlines()

## Display the raw data


In [3]:
print(len(cheatData))
cheatData[:5]

9732


['label: 2, timestamp: 1650065329, dist_s: 263.674, angle_s: 69.3421, mean_dist: 5316.49, average_mouse_speed: 0\n',
 'label: 2, timestamp: 1650065332, dist_s: 521.615, angle_s: 0, mean_dist: 5034.37, key_45: 1, average_mouse_speed: 0\n',
 'label: 2, timestamp: 1650065333, dist_s: 1155.38, angle_s: 0, mean_dist: 5892.1, key_121: 2, average_mouse_speed: 0\n',
 'label: 2, timestamp: 1650065334, dist_s: 1329.6, angle_s: 0, mean_dist: 6705.91, average_mouse_speed: 0\n',
 'label: 2, timestamp: 1650065335, dist_s: 1369.91, angle_s: 70.4558, mean_dist: 6572.16, key_45: 1, average_mouse_speed: 0\n']

## Data Preprocess


In [4]:
# define some consts
useful_key_set = [87, 65, 83, 68, 81, 69, 162, 9, 70, 71, 49, 50, 51, 52]

In [5]:
# define useful and useless key press function
def useful_key_press(keyname, value, ret):
    keynum = int(keyname[4:])
    presstime = int(value)
    if keynum in useful_key_set:
        ret['useful'] += presstime
    else:
        ret['useless'] += presstime

# for each data define a function to process it, return a dictionary
def convert_line_to_dict(record:str, keymode:str='useful')->dict:
    ret = dict()

    # default value for useful key
    if keymode == 'useful':
        ret['useful'] = 0
        ret['useless'] = 0

    features = record.split(',')
    for f in features:
        [keyname, value] = f.split(': ')
        keyname = keyname[1:] if keyname[0] == ' ' else keyname
        # timestamp
        if keyname=='timestamp':
            ret['timestamp'] = datetime.fromtimestamp(int(value))
            continue
        # average_mouse_speed
        if keyname=='average_mouse_speed':
            ret['average_mouse_speed'] = int(value)
            continue
        # dist_s
        if keyname=='dist_s':
            ret['dist_s'] = float(value)
            continue
        # angle_s
        if keyname =='angle_s':
            ret['angle_s'] = float(value)
            continue
        # mean_dist
        if keyname == 'mean_dist':
            if value == '-nan':
                ret['mean_dist'] = None
            else:
                ret['mean_dist'] = float(value)
            continue
        # key input
        if keyname[:3]=='key':
            # depends on how you handle the key
        
            # use useful and useless key record
            if keymode=='useful':
                useful_key_press(keyname, value, ret)
                continue
        # label
        if keyname=='label':
            ret['class'] = int(value)
    # if not labeled then its normal data
    # if 'class' not in ret:
    #     ret['class'] = 0
    return ret

def convert_dicts_to_dataframe(dicts, keymode:str='useful'):
    labels = list(dicts[0].keys())
    arrs = [list(d.values()) for d in dicts]
    df = pd.DataFrame(arrs, columns = labels)
    if keymode == 'useful':
        df = df[['timestamp', 'dist_s', 'angle_s', 'mean_dist', \
            'average_mouse_speed', 'useful', 'useless', 'class']]
    return df


In [6]:
dicts = [convert_line_to_dict(record) for record in cheatData]
df = convert_dicts_to_dataframe(dicts)
df

,timestamp,dist_s,angle_s,mean_dist,average_mouse_speed,useful,useless,class
0,2022-04-15 19:28:49,263.674,69.3421,5316.49,0,0,0,2
1,2022-04-15 19:28:52,521.615,0.0000,5034.37,0,0,1,2
2,2022-04-15 19:28:53,1155.380,0.0000,5892.10,0,0,2,2
3,2022-04-15 19:28:54,1329.600,0.0000,6705.91,0,0,0,2
4,2022-04-15 19:28:55,1369.910,70.4558,6572.16,0,0,1,2
...,...,...,...,...,...,...,...,...
9727,2022-04-15 22:27:02,0.000,0.0000,NaN,31,0,4,2
9728,2022-04-15 22:27:03,0.000,0.0000,NaN,10,0,2,2
9729,2022-04-15 22:27:04,0.000,0.0000,NaN,0,0,0,2
9730,2022-04-15 22:27:05,0.000,0.0000,NaN,0,0,0,2


In [7]:
df.to_csv('../csv_data/user_1.csv', index=False)